In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications import EfficientNetB0

# Verify GPU availability
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Update base_dir to the path where you uploaded your data
base_dir = 'C:\\Users\\Θάνος\\Desktop\\Thesis Thanasis\\data_aug_3'
subfolders = ['clear', 'clouds']
categories = ['Healthy_augmented', 'Damaged_augmented']
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

def load_data(base_dir, subfolders, categories, img_height, img_width):
    data = []
    labels = []
    image_paths = []
    for category in categories:
        class_num = categories.index(category)
        for subfolder in subfolders:
            folder_path = os.path.join(base_dir, subfolder, category)
            images = sorted(os.listdir(folder_path))
            for img_name in images:
                if img_name.endswith('.png'):
                    img_path = os.path.join(folder_path, img_name)
                    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
                    img_array = tf.keras.preprocessing.image.img_to_array(img)
                    data.append(img_array)
                    labels.append(class_num)
                    image_paths.append((subfolder, category, img_name))
    return np.array(data), np.array(labels), image_paths

data, labels, image_paths = load_data(base_dir, subfolders, categories, IMG_HEIGHT, IMG_WIDTH)
data = data / 255.0

# Split data ensuring twins are in the same split
def split_data(image_paths):
    unique_image_ids = list(set([img_name for subfolder, category, img_name in image_paths]))
    train_ids, test_ids = train_test_split(unique_image_ids, test_size=0.2, random_state=42)
    return train_ids, test_ids

def get_split_indices(image_paths, split_ids):
    split_indices = [i for i, (subfolder, category, img_name) in enumerate(image_paths) if img_name in split_ids]
    return split_indices

train_ids, test_ids = split_data(image_paths)
train_indices = get_split_indices(image_paths, train_ids)
test_indices = get_split_indices(image_paths, test_ids)

X_train, y_train = data[train_indices], labels[train_indices]
X_test, y_test = data[test_indices], labels[test_indices]

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test labels shape: {y_test.shape}")

# Define data augmentation with seed
def create_datagen(seed=None):
    return ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    ), seed

datagen, seed = create_datagen(seed=42)  # Set the seed for reproducibility
datagen.fit(X_train)
train_generator = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE, seed=seed)  # Use the seed here too

# Compute class weights using the training set
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

print(f"Class weights: {class_weights}")

# Fine-tune pretrained models
def fine_tune_pretrained_model(base_model, input_shape, num_classes=2):
    base_model.trainable = True  # Allow fine-tuning
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=True)
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Using EfficientNetB0 for better performance
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
model = fine_tune_pretrained_model(base_model, (IMG_HEIGHT, IMG_WIDTH, 3))

# Learning rate scheduler
def lr_schedule(epoch):
    lr = 1e-4
    if epoch > 10:
        lr *= 0.1
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

# Callbacks for training
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True, verbose=1)

# Train the model
with tf.device('/GPU:0'):
    history = model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=50,
        callbacks=[reduce_lr, early_stopping, lr_scheduler],
        class_weight=class_weights,
        verbose=1
    )

# Save the fine-tuned model
fine_tuned_model_path = 'efficientnet_fine_tuned.h5'
#model.save(fine_tuned_model_path)
print(f"EfficientNet fine-tuned model saved to {fine_tuned_model_path}")

# Evaluate the model on the test set
test_predictions = model.predict(X_test)

# Convert one-hot encoded predictions and true labels to label indices
y_test_true = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(test_predictions, axis=1)

# Generate the confusion matrix for the test set
test_conf_matrix = confusion_matrix(y_test_true, y_test_pred)

print(f"Confusion Matrix (Test) for EfficientNet:")
print(test_conf_matrix)

# Generate the classification report for the test set
test_class_report = classification_report(y_test_true, y_test_pred, target_names=categories)

print(f"Classification Report (Test) for EfficientNet:")
print(test_class_report)


TensorFlow version: 2.10.0
Num GPUs Available:  1
Training data shape: (9664, 224, 224, 3)
Test data shape: (2416, 224, 224, 3)
Training labels shape: (9664, 2)
Test labels shape: (2416, 2)
Class weights: {0: 0.8516038068382094, 1: 1.2110275689223058}
Epoch 1/50
302/302 [==============================] - 80s 217ms/step - loss: 0.6881 - accuracy: 0.5714 - lr: 1.0000e-04
Epoch 2/50
302/302 [==============================] - 70s 231ms/step - loss: 0.6460 - accuracy: 0.6279 - lr: 1.0000e-04
Epoch 3/50
302/302 [==============================] - 77s 253ms/step - loss: 0.6288 - accuracy: 0.6447 - lr: 1.0000e-04
Epoch 4/50
302/302 [==============================] - 78s 257ms/step - loss: 0.6007 - accuracy: 0.6735 - lr: 1.0000e-04
Epoch 5/50
302/302 [==============================] - 74s 244ms/step - loss: 0.5775 - accuracy: 0.6907 - lr: 1.0000e-04
Epoch 6/50
302/302 [==============================] - 74s 245ms/step - loss: 0.5390 - accuracy: 0.7201 - lr: 1.0000e-04
Epoch 7/50
302/302 [========

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications import EfficientNetB0, ResNet50, VGG16

# Verify GPU availability
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Update base_dir to the path where you uploaded your data
base_dir = 'C:\\Users\\Θάνος\\Desktop\\Thesis Thanasis\\data_aug_3'
subfolders = ['clear', 'clouds']
categories = ['Healthy_augmented', 'Damaged_augmented']
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 32

def load_data(base_dir, subfolders, categories, img_height, img_width):
    data = []
    labels = []
    image_paths = []
    for category in categories:
        class_num = categories.index(category)
        for subfolder in subfolders:
            folder_path = os.path.join(base_dir, subfolder, category)
            images = sorted(os.listdir(folder_path))
            for img_name in images:
                if img_name.endswith('.png'):
                    img_path = os.path.join(folder_path, img_name)
                    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
                    img_array = tf.keras.preprocessing.image.img_to_array(img)
                    data.append(img_array)
                    labels.append(class_num)
                    image_paths.append((subfolder, category, img_name))
    return np.array(data), np.array(labels), image_paths

data, labels, image_paths = load_data(base_dir, subfolders, categories, IMG_HEIGHT, IMG_WIDTH)
data = data / 255.0

# Split data ensuring twins are in the same split
def split_data(image_paths):
    unique_image_ids = list(set([img_name for subfolder, category, img_name in image_paths]))
    train_ids, test_ids = train_test_split(unique_image_ids, test_size=0.2, random_state=42)
    return train_ids, test_ids

def get_split_indices(image_paths, split_ids):
    split_indices = [i for i, (subfolder, category, img_name) in enumerate(image_paths) if img_name in split_ids]
    return split_indices

train_ids, test_ids = split_data(image_paths)
train_indices = get_split_indices(image_paths, train_ids)
test_indices = get_split_indices(image_paths, test_ids)

X_train, y_train = data[train_indices], labels[train_indices]
X_test, y_test = data[test_indices], labels[test_indices]

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test labels shape: {y_test.shape}")

# Define data augmentation with seed
def create_datagen(seed=None):
    return ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    ), seed

datagen, seed = create_datagen(seed=42)  # Set the seed for reproducibility
datagen.fit(X_train)
train_generator = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE, seed=seed)  # Use the seed here too

# Compute class weights using the training set
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

print(f"Class weights: {class_weights}")

# Fine-tune pretrained models
def fine_tune_pretrained_model(base_model, input_shape, num_classes=2):
    base_model.trainable = True  # Allow fine-tuning
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=True)
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Using EfficientNetB0 for better performance
base_model_efficientnet = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
model_efficientnet = fine_tune_pretrained_model(base_model_efficientnet, (IMG_HEIGHT, IMG_WIDTH, 3))

# Using ResNet50 for better performance
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
model_resnet = fine_tune_pretrained_model(base_model_resnet, (IMG_HEIGHT, IMG_WIDTH, 3))

# Using VGG16 for better performance
base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
model_vgg = fine_tune_pretrained_model(base_model_vgg, (IMG_HEIGHT, IMG_WIDTH, 3))

# Learning rate scheduler
def lr_schedule(epoch):
    lr = 1e-4
    if epoch > 10:
        lr *= 0.1
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

# Callbacks for training
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True, verbose=1)

# Train the models
with tf.device('/GPU:0'):
    history_efficientnet = model_efficientnet.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=50,
        callbacks=[reduce_lr, early_stopping, lr_scheduler],
        class_weight=class_weights,
        verbose=1
    )

    history_resnet = model_resnet.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=50,
        callbacks=[reduce_lr, early_stopping, lr_scheduler],
        class_weight=class_weights,
        verbose=1
    )

    history_vgg = model_vgg.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=50,
        callbacks=[reduce_lr, early_stopping, lr_scheduler],
        class_weight=class_weights,
        verbose=1
    )

# Save the fine-tuned models
model_efficientnet_path = 'efficientnet_fine_tuned.h5'
model_resnet_path = 'resnet_fine_tuned.h5'
model_vgg_path = 'vgg_fine_tuned.h5'

model_efficientnet.save(model_efficientnet_path)
model_resnet.save(model_resnet_path)
model_vgg.save(model_vgg_path)

print(f"EfficientNet fine-tuned model saved to {model_efficientnet_path}")
print(f"ResNet fine-tuned model saved to {model_resnet_path}")
print(f"VGG fine-tuned model saved to {model_vgg_path}")

# Evaluate the models on the test set
test_predictions_efficientnet = model_efficientnet.predict(X_test)
test_predictions_resnet = model_resnet.predict(X_test)
test_predictions_vgg = model_vgg.predict(X_test)

# Convert one-hot encoded predictions and true labels to label indices
y_test_true = np.argmax(y_test, axis=1)
y_test_pred_efficientnet = np.argmax(test_predictions_efficientnet, axis=1)
y_test_pred_resnet = np.argmax(test_predictions_resnet, axis=1)
y_test_pred_vgg = np.argmax(test_predictions_vgg, axis=1)

# Generate the confusion matrix for the test set
test_conf_matrix_efficientnet = confusion_matrix(y_test_true, y_test_pred_efficientnet)
test_conf_matrix_resnet = confusion_matrix(y_test_true, y_test_pred_resnet)
test_conf_matrix_vgg = confusion_matrix(y_test_true, y_test_pred_vgg)

print(f"Confusion Matrix (Test) for EfficientNet:")
print(test_conf_matrix_efficientnet)

print(f"Confusion Matrix (Test) for ResNet:")
print(test_conf_matrix_resnet)

print(f"Confusion Matrix (Test) for VGG:")
print(test_conf_matrix_vgg)

# Generate the classification report for the test set
test_class_report_efficientnet = classification_report(y_test_true, y_test_pred_efficientnet, target_names=categories)
test_class_report_resnet = classification_report(y_test_true, y_test_pred_resnet, target_names=categories)
test_class_report_vgg = classification_report(y_test_true, y_test_pred_vgg, target_names=categories)

print(f"Classification Report (Test) for EfficientNet:")
print(test_class_report_efficientnet)

print(f"Classification Report (Test) for ResNet:")
print(test_class_report_resnet)

print(f"Classification Report (Test) for VGG:")
print(test_class_report_vgg)


TensorFlow version: 2.10.0
Num GPUs Available:  1
Training data shape: (9664, 224, 224, 3)
Test data shape: (2416, 224, 224, 3)
Training labels shape: (9664, 2)
Test labels shape: (2416, 2)
Class weights: {0: 0.8567375886524823, 1: 1.2007952286282306}
Epoch 1/50
302/302 [==============================] - 96s 262ms/step - loss: 0.6838 - accuracy: 0.5815 - lr: 1.0000e-04
Epoch 2/50
302/302 [==============================] - 82s 269ms/step - loss: 0.6495 - accuracy: 0.6252 - lr: 1.0000e-04
Epoch 3/50
302/302 [==============================] - 86s 284ms/step - loss: 0.6233 - accuracy: 0.6447 - lr: 1.0000e-04
Epoch 4/50
302/302 [==============================] - 89s 294ms/step - loss: 0.6050 - accuracy: 0.6657 - lr: 1.0000e-04
Epoch 5/50
302/302 [==============================] - 87s 288ms/step - loss: 0.5770 - accuracy: 0.6940 - lr: 1.0000e-04
Epoch 6/50
302/302 [==============================] - 91s 300ms/step - loss: 0.5464 - accuracy: 0.7170 - lr: 1.0000e-04
Epoch 7/50
302/302 [========

TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.

In [3]:
# Evaluate the models on the test set
with tf.device('/CPU:0'):
    test_predictions_efficientnet = model_efficientnet.predict(X_test)
    test_predictions_resnet = model_resnet.predict(X_test)
    test_predictions_vgg = model_vgg.predict(X_test)

# Convert one-hot encoded predictions and true labels to label indices
y_test_true = np.argmax(y_test, axis=1)
y_test_pred_efficientnet = np.argmax(test_predictions_efficientnet, axis=1)
y_test_pred_resnet = np.argmax(test_predictions_resnet, axis=1)
y_test_pred_vgg = np.argmax(test_predictions_vgg, axis=1)

# Generate the confusion matrix for the test set
test_conf_matrix_efficientnet = confusion_matrix(y_test_true, y_test_pred_efficientnet)
test_conf_matrix_resnet = confusion_matrix(y_test_true, y_test_pred_resnet)
test_conf_matrix_vgg = confusion_matrix(y_test_true, y_test_pred_vgg)

print(f"Confusion Matrix (Test) for EfficientNet:")
print(test_conf_matrix_efficientnet)

print(f"Confusion Matrix (Test) for ResNet:")
print(test_conf_matrix_resnet)

print(f"Confusion Matrix (Test) for VGG:")
print(test_conf_matrix_vgg)

# Generate the classification report for the test set
test_class_report_efficientnet = classification_report(y_test_true, y_test_pred_efficientnet, target_names=categories)
test_class_report_resnet = classification_report(y_test_true, y_test_pred_resnet, target_names=categories)
test_class_report_vgg = classification_report(y_test_true, y_test_pred_vgg, target_names=categories)

print(f"Classification Report (Test) for EfficientNet:")
print(test_class_report_efficientnet)

print(f"Classification Report (Test) for ResNet:")
print(test_class_report_resnet)

print(f"Classification Report (Test) for VGG:")
print(test_class_report_vgg)

76/76 [==============================] - 144s 2s/step
Confusion Matrix (Test) for EfficientNet:
[[909 491]
 [534 482]]
Confusion Matrix (Test) for ResNet:
[[937 463]
 [342 674]]
Confusion Matrix (Test) for VGG:
[[   0 1400]
 [   0 1016]]
Classification Report (Test) for EfficientNet:
                   precision    recall  f1-score   support

Healthy_augmented       0.63      0.65      0.64      1400
Damaged_augmented       0.50      0.47      0.48      1016

         accuracy                           0.58      2416
        macro avg       0.56      0.56      0.56      2416
     weighted avg       0.57      0.58      0.57      2416

Classification Report (Test) for ResNet:
                   precision    recall  f1-score   support

Healthy_augmented       0.73      0.67      0.70      1400
Damaged_augmented       0.59      0.66      0.63      1016

         accuracy                           0.67      2416
        macro avg       0.66      0.67      0.66      2416
     weighted avg   

C:\Users\NickZografos\anaconda3\envs\thesis\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\NickZografos\anaconda3\envs\thesis\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\NickZografos\anaconda3\envs\thesis\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i